In [49]:
# -*- coding: utf-8 -*-
import sys
import codecs
import json

In [50]:
import requests
from bs4 import BeautifulSoup
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse as urlparse

In [51]:
link = "https://batdongsan.vn/"
t = "/ban-nha/p"
r = requests.get(link+t[0])
soup = BeautifulSoup(r.text, 'html.parser')
data = {}



In [52]:
def find_link(soup, link):
    l = []
    div_items = soup.find_all("div", attrs={"class" : "item"})
    for div_item in div_items:
        a_links = div_item.find_all('a',href=True)
        for a_link in a_links:
            l.append(a_link['href'])
    return l

In [59]:
def find_data(soup):
    data = {}

    # Tiêu đề
    titles = soup.find_all('div', {"class": "name"})
    data['title'] = [title.find('a').text.strip() for title in titles]

    # Giá
    prices = soup.find_all('span', {"class": "price"})
    data['price'] = [price.text.strip() for price in prices]

    # Diện tích
    acreages = soup.find_all('span', {"class": "acreage"})
    data['acreage'] = [acreage.text.strip() for acreage in acreages]

    # Mô tả
    descriptions = soup.find_all('div', {"class": "sapo"})
    data['description'] = [description.find('p').text.strip() for description in descriptions]

    return data

print(find_data(soup))

{'title': 'Bán nhà mặt phố Âu Cơ, Tây Hồ 40m2, MT 6m 7 tầng giá 16tỷ', 'price': '16 tỷ', 'acreage': '40m2', 'description': 'Bán nhà mặt phố Âu Cơ, Tây Hồ 40m2, MT 6m 7 tầng giá 16tỷ\r\n\r\nNHÀ MẶT PHỐ LÔ GÓC - 6 NỔI 1 HẦM THANG MÁY - THÔNG SÀN VĂN PHÒNG\r\n\r\n- Nhà...'}


In [55]:
# l = []
# i = 1
# while i < 50:
#     r = requests.get(link+t+str(i))
#     soup = BeautifulSoup(r.text, 'html.parser')
#     l.extend(find_link(soup, link))
#     i+=1

# nha = []
# for i in l:
#     re = requests.get(i)
#     s = BeautifulSoup(re.text, 'html.parser')
#     res = find_data(s)
#     nha.append(res)

# with open('bat_dong_san.json', 'a') as file:
#     json.dump(nha, file)

# print("Done")

MissingSchema: Invalid URL '#': No scheme supplied. Perhaps you meant https://#?